In [1]:
#Load in model and required packages
!pip install git+https://github.com/openai/whisper.git
!pip install pydub
!pip install git+https://github.com/huggingface/transformers
!sudo apt update && sudo apt install ffmpeg
#from google.colab import drive
import pandas as pd
import numpy as np
import whisper
from pydub import AudioSegment
import math
import string

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-qzs61_qg
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-qzs61_qg
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:0

In [32]:
Number_Of_Segments = range(len("/PATH/TO/AUDIO/SEGMENT/FOLDER/'")) #CHANGE THIS NUMBER TO MATCH YOUR NUMBER OF SEGMENTS
model = whisper.load_model("tiny.en")
transcriptions = [0,]*Number_Of_Segments
for i, _ in enumerate(transcriptions):
  print(f'{np.round((i/len(transcriptions))*100,2)}% done.')
  transcriptions[i] = model.transcribe(f'/content/segment_{i}.mp3')

In [4]:
#turn dataframe csv, download it and send to brandon
starttimes = [0,]*Number_Of_Segments
endtimes = [0,]*Number_Of_Segments
accumendtimes = [0,]*Number_Of_Segments
accumstarttimes = [0,]*Number_Of_Segments

for i in range(Number_Of_Segments):
    starttimes[i] = transcriptions[i]['segments'][0]['start']
    endtimes[i] = transcriptions[i]['segments'][-1]['end']
    accumendtimes[i] = accumendtimes[i-1] + endtimes[i]
    accumstarttimes[i] = endtimes[i-1] + accumstarttimes[i-1]

T = [0,]*Number_Of_Segments
for i,_ in enumerate(T):
  T[i] = transcriptions[i]['text']
df = pd.DataFrame({'Start Time':accumstarttimes,'End Time':accumendtimes,'Transcriptions':T})

In [24]:
splitByNumberOfMinutes = 10
startIdx = 0
endIdx = int(splitByNumberOfMinutes/2)
subdfs = [df.iloc[i:i+endIdx] for i in range(0, len(df), endIdx)]
segment_texts = []
for i in range(len(subdfs)):
  text_temp = []
  for j in range(len(subdfs[i])):
    text_temp.append(subdfs[i]['Transcriptions'].iloc[j])
  segment_texts.append(text_temp)

for i in range(len(segment_texts)):
  segment_texts[i] = " ".join(segment_texts[i])

segment_names = [f'Segment {i}' for i in range(len(segment_texts))]
segment_df = pd.DataFrame([segment_names, segment_texts],
                          index = ['Segment Names','Segment Transcription']).T
segment_df.to_csv('/PATH/WHERE/TO/SAVE/segment_df.csv')